## Step 0: Test data

In [1]:
test=  """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

source = test

## Step 0: Load the data

In [2]:
source = open("./day_7.txt").read()

## Parse the input into commands
split on the dollar sign because that indicates a new command, while mere line breaks separate input from output (e.g. for ls)

In [3]:
commands = [s.strip().splitlines() for s in source.split("$ ") if len(s.strip()) >0]
#commands

## Create a tree class that contains child trees.
Because we want to execute on the list comprehension, we treat the root as a semi-class-function. That way we run the "execute" passing reference to the top level.

In [4]:

class Tree:
    def __init__(self, parent=None):
        self.tree = {}
        self.pointer = self
        self.parent = parent
        self.dirs = {}
        self.files = {}
       # self.parent = 
    def execute(self, arr):
        commandline = arr[0]
        command = commandline.split()[0]
        if command == "cd":
            path = commandline.split()[1]
            if path == "/":
                self.pointer = self
            elif path == "..":
                self.pointer =self.pointer.parent
            else: 
                self.pointer = self.pointer.dirs[path]
        elif command == "ls":
            for item in arr[1:]:
                if item.startswith("dir"):
                    self.pointer.dirs[item[4:]] = Tree(self.pointer)
                else:
                    self.pointer.files[item.split()[1]] = int(item.split()[0])
    def size(self):
       return sum([file for file in self.files.values()]) +    sum([dir.size() for dir in self.dirs.values()])
    def recurse(self, l):
        l(self)
        for dir in self.dirs.values():
            dir.recurse(l)


## Populate the tree with the commands

In [5]:
t = Tree()
_ = [t.execute(arr) for arr in commands]

## Part 1: Get the sum of small directories
1. Find the directories with less than 100k, and sum them 

In [6]:

dirs = []
t.recurse(lambda l: dirs.append(l.size()))
#t.tree
sum([dir for dir in dirs if dir < 100000])

1447046

## Part 2: Find the smallest directory that would free up enough space.
1. Find the overage: the amount of free space we are missing that we have to recover
2. Find the smallest path in the tree that has more than the overage bytes

In [7]:
overage = 70000000 - t.size() - 30000000
dirs = []
t.recurse(lambda l: dirs.append(l.size()))
min([dir for dir in dirs if dir > (overage * -1)])

578710